In [2]:
import os

# Create the 'templates' directory if it doesn't exist
if not os.path.exists('templates'):
    os.makedirs('templates')

# Updated HTML template with threshold slider
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Stroke Prediction</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 20px;
            background: #f5f5f5;
        }
        .container {
            max-width: 600px;
            margin: 0 auto;
            padding: 20px;
            background: #fff;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        input, button, select {
            display: block;
            margin: 10px 0;
            width: 100%;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            background-color: #007bff;
            color: white;
            border: none;
            cursor: pointer;
            border-radius: 5px;
        }
        button:hover {
            background-color: #0056b3;
        }
        #result {
            margin-top: 20px;
        }
        button.print-btn {
            margin-top: 20px;
            background-color: #28a745;
        }
        button.print-btn:hover {
            background-color: #218838;
        }
        .slider-value {
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Stroke Prediction</h1>
        <form id="predictForm">
            <label for="gender">Gender:</label>
            <select id="gender" name="gender" required>
                <option value="Male">Male</option>
                <option value="Female">Female</option>
            </select>

            <label for="age">Age:</label>
            <input type="number" step="any" id="age" name="age" required>

            <label for="hypertension">Hypertension (0 = No, 1 = Yes):</label>
            <input type="number" id="hypertension" name="hypertension" min="0" max="1" required>

            <label for="heart_disease">Heart Disease (0 = No, 1 = Yes):</label>
            <input type="number" id="heart_disease" name="heart_disease" min="0" max="1" required>

            <label for="ever_married">Ever Married:</label>
            <select id="ever_married" name="ever_married" required>
                <option value="No">No</option>
                <option value="Yes">Yes</option>
            </select>

            <label for="Residence_type">Residence Type:</label>
            <select id="Residence_type" name="Residence_type" required>
                <option value="Rural">Rural</option>
                <option value="Urban">Urban</option>
            </select>

            <label for="avg_glucose_level">Average Glucose Level:</label>
            <input type="number" step="any" id="avg_glucose_level" name="avg_glucose_level" required>

            <label for="bmi">BMI:</label>
            <input type="number" step="any" id="bmi" name="bmi" required>

            <label for="work_type">Work Type:</label>
            <select id="work_type" name="work_type" required>
                <option value="Never_worked">Never Worked</option>
                <option value="Private">Private</option>
                <option value="Self-employed">Self-employed</option>
                <option value="children">Children</option>
            </select>

            <label for="smoking_status">Smoking Status:</label>
            <select id="smoking_status" name="smoking_status" required>
                <option value="formerly smoked">Formerly Smoked</option>
                <option value="never smoked">Never Smoked</option>
                <option value="smokes">Smokes</option>
            </select>

            <label for="threshold">Decision Threshold (%): <span id="thresholdValue" class="slider-value">50</span>%</label>
            <input type="range" id="threshold" name="threshold" min="1" max="100" value="50" oninput="thresholdValue.innerText=this.value">

            <button type="button" onclick="predict()">Predict</button>
        </form>
        <div id="result">
            <h2>Prediction Result:</h2>
            <p id="prediction"></p>
        </div>
        <button class="print-btn" onclick="window.print()">Print This Page</button>
    </div>
    <script>
        async function predict() {
            const form = document.getElementById('predictForm');
            const formData = new FormData(form);

            const response = await fetch('/predict', {
                method: 'POST',
                body: formData
            });
            const result = await response.json();
            document.getElementById('prediction').innerText =
                'Prediction: ' + (result.prediction || result.error) +
                ' | Stroke Probability: ' + (result.stroke_probability || 'N/A') + '%';
        }
    </script>
</body>
</html>
'''

# Save HTML template
with open('templates/index.html', 'w') as f:
    f.write(html_template)


In [3]:
import os
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import joblib
import pandas as pd

# Load preprocessing objects and model
scaler = joblib.load('scaler_strok.pkl')
ohe = joblib.load('onehot_encoder_strok.pkl')
model = joblib.load('stacking_model_stroke.pkl')

# Start Flask app
app = Flask(__name__)

# Setup Ngrok
ngrok.set_auth_token("31jBbcpMlTVUHbbIpo7tvFEnzAB_4sYYrfhqtMwYWcAxhzm")  # replace with your token
port = 5001
public_url = ngrok.connect(port)
print(f"Ngrok Tunnel URL: {public_url}")

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Collect raw inputs
        data = {
            'gender': request.form['gender'],
            'age': float(request.form['age']),
            'hypertension': int(request.form['hypertension']),
            'heart_disease': int(request.form['heart_disease']),
            'ever_married': request.form['ever_married'],
            'Residence_type': request.form['Residence_type'],
            'avg_glucose_level': float(request.form['avg_glucose_level']),
            'bmi': float(request.form['bmi']),
            'work_type': request.form['work_type'],
            'smoking_status': request.form['smoking_status']
        }
        threshold = float(request.form.get('threshold', 50)) / 100.0  # default 0.5

        df = pd.DataFrame([data])

        # Apply mapping
        df['ever_married'] = df['ever_married'].map({'No':0, 'Yes':1})
        df['gender'] = df['gender'].map({'Male':0, 'Female':1})
        df['Residence_type'] = df['Residence_type'].map({'Rural':0, 'Urban':1})

        # One-hot encode
        ohe_features = ohe.transform(df[['work_type', 'smoking_status']])
        ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(['work_type','smoking_status']))
        df = df.drop(columns=['work_type','smoking_status']).reset_index(drop=True)
        df = pd.concat([df, ohe_df], axis=1)

        # Scale numeric features
        df[['age','avg_glucose_level','bmi']] = scaler.transform(df[['age','avg_glucose_level','bmi']])

        # Predict probabilities
        proba = model.predict_proba(df)[0]
        stroke_prob = float(proba[1])
        pred = int(stroke_prob >= threshold)

        prediction_label = "Stroke" if pred == 1 else "No Stroke"

        return jsonify({
            'prediction': prediction_label,
            'stroke_probability': round(stroke_prob * 100, 2)
        })

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(port=port)


Ngrok Tunnel URL: NgrokTunnel: "https://e59b401f797c.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [02/Sep/2025 14:43:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2025 14:43:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Sep/2025 14:48:49] "POST /predict HTTP/1.1" 200 -
t=2025-09-02T15:30:03+0500 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=6e940b692a72bec5 err="failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_328ejg11hwYrtoYHm3lhreJrIb5, tn_328gvWZKCXuy9E8SfmebyqMF8UM, tn_328dTUj5ZjNRinjEGDEhRYnYqdc\n\r\n\r\nERR_NGROK_324\r\n"


In [1]:
import os

# Create the 'templates' directory if it doesn't exist
if not os.path.exists('templates'):
    os.makedirs('templates')

# Modern dark UI with Yes/No toggle buttons and result card
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>Stroke Prediction</title>
  <style>
    :root{
      --bg:#121212;
      --card:#1e1e1e;
      --text:#f1faee;
      --muted:#adb5bd;
      --accent:#e63946; /* red */
      --accent-2:#457b9d; /* blue */
      --outline:#2a2a2a;
      --success:#2ecc71;
    }
    *{box-sizing:border-box}
    body{
      margin:0; padding:0;
      font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
      background:var(--bg); color:var(--text);
    }
    header{
      padding:24px; text-align:center;
      border-bottom:1px solid var(--outline);
      background:linear-gradient(180deg, rgba(255,255,255,0.03), transparent);
    }
    header h1{ margin:0; font-size:28px; letter-spacing:0.3px; }
    header p{ margin:6px 0 0; color:var(--muted); font-size:14px; }

    .container{
      max-width:860px; margin:32px auto; padding:0 16px;
      display:grid; grid-template-columns: 1fr;
      gap:16px;
    }
    .card{
      background:var(--card); border:1px solid var(--outline);
      border-radius:16px; padding:20px;
      box-shadow: 0 6px 24px rgba(0,0,0,0.25);
    }
    .grid{
      display:grid; gap:16px;
      grid-template-columns: repeat(auto-fit, minmax(240px, 1fr));
    }
    label{ display:block; margin-bottom:6px; color:var(--muted); font-size:13px; }
    input[type="number"], select{
      width:100%; padding:12px 12px;
      border-radius:12px; border:1px solid var(--outline);
      background:#161616; color:var(--text);
      outline:none; transition:border-color .2s ease, box-shadow .2s ease;
    }
    input[type="number"]:focus, select:focus{
      border-color:var(--accent-2);
      box-shadow:0 0 0 3px rgba(69,123,157,0.25);
    }
    .btn{
      display:inline-flex; align-items:center; justify-content:center;
      padding:12px 16px; border-radius:12px; border:1px solid var(--outline);
      background:#161616; color:var(--text);
      cursor:pointer; user-select:none; transition:all .2s ease;
    }
    .btn:hover{ border-color:var(--accent-2); }
    .btn.primary{ background:var(--accent-2); border-color:var(--accent-2); }
    .btn.primary:hover{ filter:brightness(1.05); }
    .btn.ghost{ background:#161616; }

    /* Toggle group (Yes/No) */
    .toggle-group{ display:flex; gap:8px; }
    .toggle{
      flex:1; text-align:center;
      padding:10px 12px; border-radius:12px; border:1px solid var(--outline);
      background:#161616; color:var(--text); cursor:pointer;
      transition:all .2s ease;
    }
    .toggle:hover{ border-color:var(--accent-2); }
    .toggle.active.yes{ background:var(--accent); border-color:var(--accent); }
    .toggle.active.no{ background:var(--accent-2); border-color:var(--accent-2); }
    .hint{ color:var(--muted); font-size:12px; margin-top:4px; }

    /* Result card */
    .result-wrap{ display:none; }
    .result-card{
      margin-top:8px;
      border-radius:16px; padding:18px;
      border:1px solid var(--outline);
      background:#141414;
      display:flex; align-items:center; justify-content:space-between; gap:16px;
    }
    .result-badge{
      padding:8px 12px; border-radius:999px;
      font-weight:600; letter-spacing:.4px;
    }
    .result-positive{ /* Stroke */
      background:rgba(230,57,70,0.15); color:#ff7b86; border:1px solid rgba(230,57,70,0.35);
    }
    .result-negative{ /* No Stroke */
      background:rgba(46,204,113,0.12); color:#7fe2a5; border:1px solid rgba(46,204,113,0.35);
    }

    footer{
      margin-top:24px; padding:20px 16px; text-align:center;
      color:var(--muted); font-size:13px;
      border-top:1px solid var(--outline);
      background:linear-gradient(0deg, rgba(255,255,255,0.03), transparent);
    }
    a.inline{ color:#a8d2ff; text-decoration:none; }
    a.inline:hover{ text-decoration:underline; }
  </style>
</head>
<body>
  <header>
    <h1>Predicting Stroke Susceptibility Through AI Models</h1>
    <p>Modern, professional UI — enter details to predict stroke risk (Yes/No)</p>
  </header>

  <main class="container">
    <section class="card">
      <form id="predictForm">
        <div class="grid">
          <div>
            <label for="gender">Gender</label>
            <select id="gender" name="gender" required>
              <option value="Male">Male</option>
              <option value="Female">Female</option>
            </select>
          </div>

          <div>
            <label for="age">Age</label>
            <input type="number" step="any" id="age" name="age" required>
            <div class="hint">Use integers or decimals</div>
          </div>

          <!-- Hypertension Yes/No -->
          <div>
            <label>Hypertension</label>
            <div class="toggle-group" data-target="hypertension">
              <div class="toggle no active" data-value="0">No</div>
              <div class="toggle yes" data-value="1">Yes</div>
            </div>
            <input type="hidden" id="hypertension" name="hypertension" value="0">
          </div>

          <!-- Heart Disease Yes/No -->
          <div>
            <label>Heart Disease</label>
            <div class="toggle-group" data-target="heart_disease">
              <div class="toggle no active" data-value="0">No</div>
              <div class="toggle yes" data-value="1">Yes</div>
            </div>
            <input type="hidden" id="heart_disease" name="heart_disease" value="0">
          </div>

          <!-- Ever Married Yes/No -->
          <div>
            <label>Ever Married</label>
            <div class="toggle-group" data-target="ever_married">
              <div class="toggle no active" data-value="0">No</div>
              <div class="toggle yes" data-value="1">Yes</div>
            </div>
            <input type="hidden" id="ever_married" name="ever_married" value="0">
          </div>

          <div>
            <label for="Residence_type">Residence Type</label>
            <select id="Residence_type" name="Residence_type" required>
              <option value="Rural">Rural</option>
              <option value="Urban">Urban</option>
            </select>
          </div>

          <div>
            <label for="avg_glucose_level">Average Glucose Level</label>
            <input type="number" step="any" id="avg_glucose_level" name="avg_glucose_level" required>
          </div>

          <div>
            <label for="bmi">BMI</label>
            <input type="number" step="any" id="bmi" name="bmi" required>
          </div>

          <div>
            <label for="work_type">Work Type</label>
            <select id="work_type" name="work_type" required>
              <option value="Never_worked">Never Worked</option>
              <option value="Private">Private</option>
              <option value="Self-employed">Self-employed</option>
              <option value="children">Children</option>
            </select>
          </div>

          <div>
            <label for="smoking_status">Smoking Status</label>
            <select id="smoking_status" name="smoking_status" required>
              <option value="formerly smoked">Formerly Smoked</option>
              <option value="never smoked">Never Smoked</option>
              <option value="smokes">Smokes</option>
            </select>
          </div>
        </div>

        <div style="margin-top:16px; display:flex; gap:10px; justify-content:flex-end;">
          <button type="button" class="btn ghost" onclick="document.getElementById('predictForm').reset(); resetToggles();">Reset</button>
          <button type="button" class="btn primary" onclick="predict()">Predict</button>
        </div>
      </form>

      <!-- Result (hidden until first prediction) -->
      <div id="resultWrap" class="result-wrap">
        <div id="resultCard" class="result-card">
          <div>
            <div style="font-size:14px; color:var(--muted);">Prediction Result</div>
            <div id="predictionText" style="margin-top:6px; font-size:20px; font-weight:600;"></div>
          </div>
          <div id="predictionBadge" class="result-badge">—</div>
        </div>
      </div>
    </section>
  </main>

  <footer>
    © <span id="year"></span> Predicting Stroke Susceptibility Through AI Models — All rights reserved.
  </footer>

  <script>
    // Footer year
    document.getElementById('year').innerText = new Date().getFullYear();

    // Toggle Yes/No groups -> set hidden input 0/1 and visual state
    function initToggleGroups(){
      document.querySelectorAll('.toggle-group').forEach(group=>{
        const targetName = group.getAttribute('data-target');
        const hidden = document.getElementById(targetName);
        group.querySelectorAll('.toggle').forEach(btn=>{
          btn.addEventListener('click', ()=>{
            group.querySelectorAll('.toggle').forEach(b=>b.classList.remove('active'));
            btn.classList.add('active');
            // keep stylistic class yes/no
            group.querySelectorAll('.toggle').forEach(b=>{
              b.classList.remove('yes','no');
              b.classList.add(b.textContent.trim().toLowerCase()==='yes' ? 'yes' : 'no');
            });
            hidden.value = btn.getAttribute('data-value'); // "0" or "1"
          });
        });
        // ensure initial classes are applied
        group.querySelectorAll('.toggle').forEach(b=>{
          b.classList.remove('yes','no');
          b.classList.add(b.textContent.trim().toLowerCase()==='yes' ? 'yes' : 'no');
        });
      });
    }
    function resetToggles(){
      document.querySelectorAll('.toggle-group').forEach(group=>{
        const hidden = document.getElementById(group.getAttribute('data-target'));
        const noBtn = group.querySelector('.toggle.no');
        group.querySelectorAll('.toggle').forEach(b=>b.classList.remove('active'));
        noBtn.classList.add('active');
        hidden.value = '0';
      });
    }
    initToggleGroups();

    async function predict(){
      const form = document.getElementById('predictForm');
      const formData = new FormData(form);

      const response = await fetch('/predict', {
        method: 'POST',
        body: formData
      });
      const result = await response.json();

      const wrap = document.getElementById('resultWrap');
      const card = document.getElementById('resultCard');
      const text = document.getElementById('predictionText');
      const badge = document.getElementById('predictionBadge');

      if(result && result.prediction){
        const isStroke = result.prediction.toLowerCase().includes('stroke') && !result.prediction.toLowerCase().includes('no');
        text.textContent = result.prediction;

        // badge styling
        badge.textContent = isStroke ? 'Stroke' : 'No Stroke';
        badge.classList.remove('result-positive','result-negative');
        badge.classList.add(isStroke ? 'result-positive' : 'result-negative');

        wrap.style.display = 'block';
      }else{
        text.textContent = 'Error';
        badge.textContent = '—';
        wrap.style.display = 'block';
      }
    }
  </script>
</body>
</html>
'''

# Save HTML template
with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(html_template)


In [2]:
import os
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import joblib
import pandas as pd

# Load preprocessing objects and model
scaler = joblib.load('scaler_strok.pkl')
ohe = joblib.load('onehot_encoder_strok.pkl')
model = joblib.load('stacking_model_stroke.pkl')

# Start Flask app
app = Flask(__name__)

# Setup Ngrok
ngrok.set_auth_token("Token")  # <-- replace with your token
port = 5001
public_url = ngrok.connect(port)
print(f"Ngrok Tunnel URL: {public_url}")

# Decision threshold (same functionality as before)
THRESHOLD = 0.39  # 39%

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Collect inputs (Yes/No toggle buttons provide "0" or "1" for these three fields)
        # Keep others as before
        gender = request.form['gender']                    # "Male"/"Female"
        age = float(request.form['age'])
        hypertension_raw = request.form['hypertension']    # "0"/"1"
        heart_disease_raw = request.form['heart_disease']  # "0"/"1"
        ever_married_raw = request.form['ever_married']    # "0"/"1" (from new toggles)
        residence = request.form['Residence_type']         # "Rural"/"Urban"
        avg_glucose = float(request.form['avg_glucose_level'])
        bmi = float(request.form['bmi'])
        work_type = request.form['work_type']
        smoking_status = request.form['smoking_status']

        # Coerce 0/1 safely (and also tolerate legacy "No"/"Yes" if ever posted)
        def to01(x, yes_no=False):
            x = str(x).strip()
            if x in ('0','1'):
                return int(x)
            if yes_no:  # only if we ever receive "No"/"Yes"
                return 1 if x.lower()=='yes' else 0
            # default
            return 0

        hypertension = to01(hypertension_raw)
        heart_disease = to01(heart_disease_raw)
        ever_married = to01(ever_married_raw)  # already 0/1 from UI

        # Build dataframe
        df = pd.DataFrame([{
            'gender': gender,
            'age': age,
            'hypertension': hypertension,
            'heart_disease': heart_disease,
            'ever_married': ever_married,       # already numeric
            'Residence_type': residence,
            'avg_glucose_level': avg_glucose,
            'bmi': bmi,
            'work_type': work_type,
            'smoking_status': smoking_status
        }])

        # Map remaining categoricals as in training
        df['gender'] = df['gender'].map({'Male':0, 'Female':1})
        df['Residence_type'] = df['Residence_type'].map({'Rural':0, 'Urban':1})

        # One-hot encode (same encoder you saved)
        ohe_features = ohe.transform(df[['work_type', 'smoking_status']])
        ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(['work_type','smoking_status']))

        df = df.drop(columns=['work_type','smoking_status']).reset_index(drop=True)
        df = pd.concat([df, ohe_df], axis=1)

        # Scale numeric features (same scaler you saved)
        df[['age','avg_glucose_level','bmi']] = scaler.transform(df[['age','avg_glucose_level','bmi']])

        # Predict probability and apply threshold -> Yes/No only
        proba = float(model.predict_proba(df)[0][1])
        prediction = "Patient is at Risk of Stroke" if proba >= THRESHOLD else "Patient is Not at Risk of Stroke"

        return jsonify({'prediction': prediction})

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(port=port)


Ngrok Tunnel URL: NgrokTunnel: "https://20f1b612b843.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [02/Sep/2025 15:36:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2025 15:36:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Sep/2025 15:37:56] "POST /predict HTTP/1.1" 200 -
